<img src="../../SAR_Training/English/Master/NotebookAddons/blackboard-banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>Flood Mapping using a Convolutional Neural Network (CNN)</b><img style="padding: 7px" src="../../SAR_Training/English/Master/NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/></font>

<br>
<font size="4"> <b> Alex Lewandowski; University of Alaska Fairbanks
<br>
    Adapted from ASF's AI_Water project: <br>
    McKade Sorensen, George Meier, and Rohan Weeden</b> <br>
</font>

<font size="3">This takes a prepared stack of RTC products as input, containing both VV and VH polarities (see the Prepare_Data_Stack notebook). It uses a fully convolutional neural network to create predicted water masks for all image pairs (VV and VH) in the stack.
<br><br>
ASF's AI_Water is open source and freely available on github: <b><a href="https://github.com/asfadmin/AI_Water" target="_blank">https://github.com/asfadmin/AI_Water</a></b>
<br><br>
    <font color='red'><b>Important:</b> The AI_Water neural network was trained on data collected during warm months. It was not trained on data collected from times and places experiencing sub-freezing temperatures.</font>
</font></font>
<hr>

In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/machine_learning':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "machine_learning" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "machine_learning" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "machine_learning" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

# Install Tensorflow, Keras, and other Needed Python Libraries

<font face="Calibri" size="3"> Note: this must be done only once each time your OpneSARlab server is restarted </font>

<font face="Calibri" size="3"><b>Import necessary packages and libraries</b></font>

In [ ]:
import os

from osgeo import gdal
from typing import Tuple
import numpy as np

from keras.models import Model
from keras.models import load_model as kload_model

import asf_notebook as asfn

# Setting Up Network and Path to Data Sets 

<font face="Calibri" size="3"><b>Define the path to our network</b></font>

In [ ]:
ai_water_path = '/home/jovyan/notebooks/ASF/Projects'

<font face="Calibri" size="3"><b>Write a function to create a list of paths to all tiffs in a directory</b></font>

In [ ]:
def get_tiff_paths(paths: str) -> list:
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

<font face="Calibri" size="3"><b>Enter the path to the data stack</b></font>

In [ ]:
while True:
    print("Enter the absolute path to the directory holding your tiffs.")
    tiff_dir = input()
    paths = f"{tiff_dir}/*.tif*"
    if os.path.exists(tiff_dir):
        tiff_paths = get_tiff_paths(paths)
        if len(tiff_paths) < 1:
            print(f"{tiff_dir} exists but contains no tifs.")
            print("You will not be able to proceed until tifs are prepared.")
        break
    else:
        print(f"\n{tiff_dir} does not exist.")
        continue

<font face="Calibri" size="3"><b>Move into the parent directory of the directory containing the data and create a directory in which to store the water masks</b></font>

In [ ]:
analysis_directory = os.path.dirname(tiff_dir)
os.chdir(analysis_directory)
mask_directory = f'{analysis_directory}/AI_Water_Masks'
asfn.new_directory(mask_directory)
print(f"Current working directory: {os.getcwd()}")

# Discover Available Data Sets

<font face="Calibri" size="3"><b>Write a function to create a dictionary containing lists of each vv/vh pair</b></font>

In [ ]:
def group_polarizations(tiff_paths: list) -> dict:
    pths = {}
    for tiff in tiff_paths:
        product_name = tiff.split('.')[0][:-2]
        if product_name in pths:
            pths[product_name].append(tiff)
        else:
            pths.update({product_name: [tiff]})
            pths[product_name].sort()
    return pths

<font face="Calibri" size="3"><b>Write a function to confirm the presence of both VV and VH images in all image sets</b></font>

In [ ]:
def confirm_dual_polarizations(paths: dict) -> bool:
    for p in paths:
        if len(paths[p]) == 2:
            if ('vv' not in paths[p][1] and 'VV' not in paths[p][1]) or \
            ('vh' not in paths[p][0] and 'VH' not in paths[p][0]):
                return False
    return True   

<font face="Calibri" size="3"><b>Create a dictionary of VV/VH pairs and check it for completeness</b></font>

In [ ]:
grouped_pths = group_polarizations(tiff_paths)
if not confirm_dual_polarizations(grouped_pths):
    print("ERROR: AI_Water requires both VV and VH polarizations.")
else:
    print("Confirmed presence of VV and VH polarities for each product.")
    
#print(grouped_pths) #uncomment to print VV/VH path pairs

# Creating Some Helper Scripts

<font face="Calibri" size="3"><b>Write a function to pad an image, so it may be split into tiles with consistent dimensions</b></font>

In [ ]:
def pad_image(image: np.ndarray, to: int) -> np.ndarray:
    height, width = image.shape

    n_rows, n_cols = get_tile_row_col_count(height, width, to)
    new_height = n_rows * to
    new_width = n_cols * to

    padded = np.zeros((new_height, new_width))
    padded[:image.shape[0], :image.shape[1]] = image
    return padded

<font face="Calibri" size="3"><b>Write a function to tile an image</b></font>

In [ ]:
def tile_image(image: np.ndarray, width: int = 512, height: int = 512) -> np.ndarray:
    _nrows, _ncols = image.shape
    _strides = image.strides

    nrows, _m = divmod(_nrows, height)
    ncols, _n = divmod(_ncols, width)

    assert _m == 0, "Image must be evenly tileable. Please pad it first"
    assert _n == 0, "Image must be evenly tileable. Please pad it first"

    return np.lib.stride_tricks.as_strided(
        np.ravel(image),
        shape=(nrows, ncols, height, width),
        strides=(height * _strides[0], width * _strides[1], *_strides),
        writeable=False
    ).reshape(nrows * ncols, height, width)


<font face="Calibri" size="3"><b>Write a function to calculate the number of rows and columns of tiles needed to tile an image to a given size</b></font>

In [ ]:
def get_tile_row_col_count(height: int, width: int, tile_size: int) -> Tuple[int, int]:
    return int(np.ceil(height / tile_size)), int(np.ceil(width / tile_size))

<font face="Calibri" size="3"><b>Write a function to load a trained model</b></font>

In [ ]:
def load_model(model_path: str) -> Model:
    """ Loads and returns a model. Attaches the model name and that model's
    history. """
    model_dir = os.path.dirname(model_path)
    print(f"model_dir: {model_dir}")
    model = kload_model(model_path)

    # Attach our extra data to the model
    model.__asf_model_name = model_path

    return model

<font face="Calibri" size="3"><b>Write a function to save a mask</b></font>

In [ ]:
def write_mask_to_file(mask: np.ndarray, file_name: str, projection: str, geo_transform: str) -> None:
    (width, height) = mask.shape
    out_image = gdal.GetDriverByName('GTiff').Create(
        file_name, height, width, bands=1
    )
    out_image.SetProjection(projection)
    out_image.SetGeoTransform(geo_transform)
    out_image.GetRasterBand(1).WriteArray(mask)
    out_image.GetRasterBand(1).SetNoDataValue(0)
    out_image.FlushCache()

# Run CNN-based Flood Mapping on Discovered Data

<font face="Calibri" size="3"><b>Load the AI_Water model and print a summary of its fully convolutional neural network architecture</b></font>

In [ ]:
model_path = f'{ai_water_path}/network.h5'
model = load_model(model_path)
print(model.summary())

<font face="Calibri" size="3"><b>Iterate through each VV/VH pair, using AI_Water to create a predicted water mask for each</b></font>

In [ ]:
for pair in grouped_pths:
    for tiff in grouped_pths[pair]:
        f = gdal.Open(tiff)
        img_array = f.ReadAsArray()
        original_shape = img_array.shape
        n_rows, n_cols = get_tile_row_col_count(*original_shape, tile_size=512)
        print(f'tiff: {tiff}')
        if 'vv' in tiff or 'VV' in tiff:
            vv_array = pad_image(f.ReadAsArray(), 512)
            invalid_pixels = np.nonzero(vv_array == 0.0)
            vv_tiles = tile_image(vv_array)
        else:
            vh_array = pad_image(f.ReadAsArray(), 512)
            invalid_pixels = np.nonzero(vh_array == 0.0)
            vh_tiles = tile_image(vh_array)
            
    # Predict masks
    masks = model.predict(
        np.stack((vh_tiles, vv_tiles), axis=3), batch_size=1, verbose=1
    )
    masks.round(decimals=0, out=masks)
    # Stitch masks together
    mask = masks.reshape((n_rows, n_cols, 512, 512)) \
                .swapaxes(1, 2) \
                .reshape(n_rows * 512, n_cols * 512)  # yapf: disable

    mask[invalid_pixels] = 0
    filename, ext = os.path.basename(tiff).split('.')
    outfile = f"{mask_directory}/{filename[:-3]}_water_mask.{ext}"
    write_mask_to_file(mask, outfile, f.GetProjection(), f.GetGeoTransform())

# Version Log

<font face="Calibri" size="2" color="gray"> <i> AI_Water_Masks_From_Prepared_Data_Stack.ipynb - Version 1.2.0 - April 2021
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>namespace asf_notebook</li>
    </ul>
    </i>
</font>